In [35]:
import cv2
from skimage import measure, color
import matplotlib.pyplot as plt

In [7]:
testRect = {
    "audit_type": "rect",
    "id": 27,
    "page_number": 1,
    "template_signatory_id": 10,
    "x1": 0.122898037659841,
    "x2": 0.169011505478072,
    "y1": 0.912442396313364,
    "y2": 0.943205053865169
}

In [9]:
source_image = cv2.imread("test_files/VacantLandContract - Signed [1] (1)-0.png")


In [10]:
source_image.shape

(792, 612, 3)

In [90]:
img = cv2.cvtColor(source_image, cv2.COLOR_BGR2GRAY)

In [11]:
def translate_coords(coord1, coord2, span):
    return [int(coord1 * span), int(coord2 * span)]
    

In [16]:
def unsharp_mask(image):
    """Soften an input image.

    Parameters
    ----------
    image : numpy ndarray
        The input image.

    Returns
    -------
    numpy ndarray
        The soften image.

    """
    # perform GaussianBlur filter to use it in unsharpening mask
    gaussian_3 = cv2.GaussianBlur(image, (9, 9), 10.0)
    # calculates the weighted sum of two arrays (source image and GaussianBlur
    # filter) to perform unsharpening mask
    unsharp_image = cv2.addWeighted(image, 1.5, gaussian_3, -0.5, 0, image)
    # return unsharpened image
    return unsharp_image

In [65]:
cropped = img[slice(*translate_coords(testRect['y1'], testRect['y2'], source_image.shape[0])), slice(*translate_coords(testRect['x1'], testRect['x2'], source_image.shape[1]))]

In [66]:
cropped.shape

(25, 28)

In [67]:
cv2.imwrite("output_files/Step One Cropped.png", cropped)

True

In [68]:
sharpened = unsharp_mask(cropped)

In [69]:
cv2.imwrite("output_files/Step Two Sharpened.png", sharpened)

True

In [92]:
img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)[1]

In [93]:
blobs = img > img.mean()

In [94]:
blobs_labels = measure.label(blobs, background=1)

In [95]:
image_label_overlay = color.label2rgb(blobs_labels, image=img)

In [96]:
img.shape

(792, 612)

In [97]:
ax.imshow(image_label_overlay)
ax.set_axis_off()
plt.tight_layout()
fig.show()

/var/folders/6x/1jtnwm815415bccq7hd68fvh0000gn/T/ipykernel_48736/3476875622.py:4: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  fig.show()


<Figure size 640x480 with 0 Axes>

In [80]:
plt

<module 'matplotlib.pyplot' from '/Users/mikekilmer/.pyenv/versions/3.12.1/envs/pfsignatures/lib/python3.12/site-packages/matplotlib/pyplot.py'>

In [53]:
the_biggest_component = 0
total_area = 0
counter = 0
average = 0.0
for region in measure.regionprops(blobs_labels):
		if (region.area > 10):
				total_area = total_area + region.area
				counter = counter + 1
		# print region.area # (for debugging)
		# take regions with large enough areas
		if (region.area >= 10):
				if (region.area > the_biggest_component):
						the_biggest_component = region.area

In [54]:
the_biggest_component

357.0